In [ ]:
# import libraries
import os
import re
import sys
import math
import pandas as pd
from urllib.request import urlopen, Request
from urllib.parse import quote_plus
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import timedelta
from random import randint
from time import sleep
from unicodedata import normalize

# set user agent
#ua = str(UserAgent().random)
ua = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'

# get page numbers
def getPageNums(sDate):
    main = "https://search.naver.com/search.naver?where=post"
    sub0 = "&sm=tab_pge"
    sub1 = "&query=" + quote_plus(keywd)
    sub2 = "&st=date"
    sub3 = "&date_option=8"
    sub4 = "&date_from=" + sDate
    sub5 = "&date_to=" + sDate
    sub6 = "&dup_remove=1"
    sub7 = "&srchby=all"
    sub8 = "&ie=utf8"
    sub9 = "&start=1"
    
    URL = main + sub0 + sub1 + sub2 + sub3 + sub4 + sub5 + sub6 + sub7 + sub8 + sub9
    #print(URL)
    
    resp = Request(URL, data=None, headers={'User-Agent':ua})
    html = urlopen(resp)
    soup = BeautifulSoup(html, 'html.parser')
    
    # check page number
    try:
        count = soup.find('span',{'class':'title_num'}).get_text().split('/')[1].strip().replace("건","").replace(",","")
        count = int(count)
    except:
        count = 0
        pass
        
    print("The number of blogs for", sDate, ":", count)
    
    return(count)


# get blog links and summaries
def getBlogLinks(sDate, count): 
    sumLinks = pd.DataFrame()
    maxPages = math.ceil(count/10)
        
    for i in range(maxPages):
        page = i*10 + 1
        #print(page)
        
        main = "https://search.naver.com/search.naver?where=post"
        sub0 = "&sm=tab_pge"
        sub1 = "&query=" + quote_plus(keywd)
        sub2 = "&st=date"
        sub3 = "&date_option=8"
        sub4 = "&date_from=" + sDate
        sub5 = "&date_to=" + sDate
        sub6 = "&dup_remove=1"
        sub7 = "&srchby=all"
        sub8 = "&ie=utf8"
        sub9 = "&start=" + str(page)
        
        URL = main + sub0 + sub1 + sub2 + sub3 + sub4 + sub5 + sub6 + sub7 + sub8 + sub9
        #print(URL)
        
        resp = Request(URL, data=None, headers={'User-Agent':ua})
        html = urlopen(resp)
        soup = BeautifulSoup(html, 'html.parser')
        
        blogs = soup.findAll("li",{"class":"sh_blog_top"})
        
        title = []
        pdate = []
        link = []
        text = []
        #bloger = []
        
        for blog in blogs:
            try:
                title.append(blog.find('dt').find('a').get_text())
                pdate.append(blog.find('dd',{'class':'txt_inline'}).get_text().split(" ")[0])
                link.append(blog.find('dt').find('a').attrs['href'].replace("?Redirect=Log&logNo=","/"))
                text.append(blog.find('dd',{'class':'sh_blog_passage'}).get_text())
                #bloger.append(blog.find('a',{'class':'txt84'}).get_text())
            except:
                pass
        
        df = pd.DataFrame({"keyword":keywd, "title":title, "postdate":pdate, "link":link, "summary":text})
        sumLinks = sumLinks.append(df)
    
    return sumLinks


# input query and dates
keywd = str(input("검색어:"))
strDate = str(input("검색 시작일(YYYYMMDD):"))
endDate = str(input("검색 종료일(YYYYMMDD):"))
print("-----------------------------------")

# calculate timedelta
dateGap = datetime.strptime(endDate, '%Y%m%d') - datetime.strptime(strDate, '%Y%m%d')
dateGap.days + 1

# collect blogs information
result = pd.DataFrame()

for i in range(dateGap.days+1):
    sDate = datetime.strptime(strDate, '%Y%m%d') + timedelta(days=i)
    sDate = sDate.strftime("%Y%m%d")
    
    # get blog page numbers
    count = getPageNums(sDate)
    
    # get blog links and summary
    summary = getBlogLinks(sDate, count)
    
    # rearragne columns
    #summary = summary[["keyword", "title", "postdate", "link", "summary"]]
    
    # add by rows
    result = result.append(summary)
    
    
# check blog numbers before removing duplicated
print("\n")
print("The total number of blogs :", len(result))

# remove duplicated
result = result.reset_index(drop=True)
result = result.drop_duplicates(['link'], keep='last')
print("After removing duplicated :", len(result))
print("\n")


# get full text from various blogs
def getTextFromNaver(url):
    try:
        resp = Request(url, data=None, headers={'User-Agent':ua})
        html = urlopen(resp)
        soup = BeautifulSoup(html, 'html.parser')
        
        try:
            url1 = soup.find('frame',{'id':'screenFrame'}).attrs['src'].split("?")[0]
            url1
        except:
            url1 = url
        
        postNo = url1.split("/")[-1]
        
        html = urlopen(url1)
        soup = BeautifulSoup(html, 'html.parser')
        soup.find("frame",{"id":"mainFrame"}).attrs["src"]
        url2 = "http://blog.naver.com" + soup.find("frame",{"id":"mainFrame"}).attrs["src"]
        
        html = urlopen(url2)
        soup = BeautifulSoup(html, 'html.parser')
        
        text = soup.find('div',{'id':'post-view' + postNo}).get_text()
        full = re.compile('[^ㄱ-ㅣ가-힣]+').sub(' ', text).strip()
        return(full)
    except:
        pass


def getTextFromEgloos(url):
    try:
        resp = Request(url, data=None, headers={'User-Agent':ua})
        html = urlopen(resp)
        soup = BeautifulSoup(html, 'html.parser')
        
        text = soup.find('div',{'class':'hentry'}).get_text()
        full = re.compile('[^ㄱ-ㅣ가-힣]+').sub(' ', text).strip()
        return(full)
    except:
        pass


def getTextFromTistory(url):
    try:
        resp = Request(url, data=None, headers={'User-Agent':ua})
        html = urlopen(resp)
        soup = BeautifulSoup(html, 'html.parser')
        
        try:
            text = soup.find('div',{'class':'article'}).get_text()
        except:
            text = soup.find('div',{'class':'area_view'}).get_text()
        full = re.compile('[^ㄱ-ㅣ가-힣]+').sub(' ', text).strip()
        return(full)
    except:
        pass


def getTextFromDaum(url):
    try:
        resp = Request(url, data=None, headers={'User-Agent':ua})
        html = urlopen(resp)
        soup = BeautifulSoup(html, 'html.parser')
        
        url1 = 'http://blog.daum.net' + soup.find('frame',{'name':'BlogMain'}).attrs['src']
        html = urlopen(url1)
        soup = BeautifulSoup(html, 'html.parser')
        
        url2 = 'http://blog.daum.net' + soup.find('div',{'id':'cContentBody'}).find('iframe').attrs['src']
        html = urlopen(url2)
        soup = BeautifulSoup(html, 'html.parser')
        
        text = soup.find('div',{'id':'contentDiv'}).get_text()
        full = re.compile("[^ㄱ-ㅣ가-힣]+").sub(' ', text).strip()
        return(full)
    except:
        pass


# get full bloger titles
def getBlogerFromTistory(url):
    try:
        resp = Request(url, data=None, headers={'User-Agent':ua})
        html = urlopen(resp)
        soup = BeautifulSoup(html, 'html.parser')
        bloger = soup.find("meta",{"property":"og:site_name"}).attrs["content"]
    except:
        pass
    
    return(bloger)


def getBlogerFromOthers(url):
    try:
        resp = Request(url, data=None, headers={'User-Agent':ua})
        html = urlopen(resp)
        soup = BeautifulSoup(html, 'html.parser')
        bloger = soup.find("title").get_text().split(":")[0].strip()
    except:
        pass
    
    return(bloger)


# create user defined function for checking the domain of a blog
def getBlogDomain(url):
    try:
        resp = Request(url, data=None, headers={'User-Agent':ua})
        html = urlopen(resp)
        soup = BeautifulSoup(html, 'html.parser')
    except:
        pass
    
    domain = ''
    
    # for egloos or tistory    
    try:
        domain = soup.find('link',{'rel':'shortcut icon'}).attrs['href']
    except:
        pass
    
    # for daum
    try:
        domain = soup.find('link',{'rel':'icon'}).attrs['href']
    except:
        pass
    
    # for naver or blog.me
    try:
        domain = soup.find('link',{'rel':'wlwmanifest'}).attrs['href']
    except:
        pass
    
    return(domain)


# set urls
urls = result["link"]

# set default dataframes
df = pd.DataFrame()
fulltext = pd.DataFrame()
i = 0

# fetch full text from blogs
for url in urls:
    i += 1
    print(i, url)
    
    # set time sleep randomly
    sleep(randint(1,3))

    domain = getBlogDomain(url)
    
    if 'naver' in domain:
        print("naver blog!")
        full = getTextFromNaver(url)
        bloger = getBlogerFromOthers(url)
        
    elif 'blog.me' in domain:
        print("naver blog!")
        full = getTextFromNaver(url)
        bloger = getBlogerFromOthers(url)
        
    elif 'tistory' in domain:
        print("tistory blog!")
        full = getTextFromTistory(url)
        bloger = getBlogerFromTistory(url)
        
    elif 'egloos' in domain:
        print("egloos blog!")
        full = getTextFromEgloos(url)
        bloger = getBlogerFromOthers(url)
        
    elif 'daum' in domain:
        print("daum blog!")
        full = getTextFromDaum(url)
        bloger = getBlogerFromOthers(url)
    
    else:
        print("Next!")
        full = ''
    
    df = pd.DataFrame({"link":[url], "fulltext":[full], "bloger":[bloger]})
    fulltext = fulltext.append(df)

print("It's done!")


# create the final dataframe 
final = pd.merge(result, fulltext, how="left", on="link", left_index=True)
final = final.reset_index(drop=True)
final = final[["keyword", "title", "postdate", "link", "summary", "bloger", "fulltext"]]
print("The number of blogs:", len(final))

# download it as an Excel file
wdir = "./Data/"
if not os.path.exists(wdir):
    os.makedirs(wdir)

wtime = datetime.today().strftime("%Y%m%d")
fname = wdir + "Blog_" + normalize("NFC", keywd) + "_" + strDate + "_" + endDate + ".xlsx"
final.to_excel(fname, index=False)